<a href="https://colab.research.google.com/github/alexswcr/Monopoly-AI-Training/blob/Add-Files/Human_Simulated_Tycoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Monopoly Agent using a Genetic Algorithm Approach

~This notebook was made to create an environment where autonomous player agents would be able to train their parameters to be able to make optimal decisions during the game.~ This is the human player version, you can go to Runtime at the top left and press 'Run All' to start a game of monopoly :)


It was important to create a separate fully functioning game of monopoly as the main version of the game in Unity would take very long to play (around 4 hours per game) due to animations and other graphical elements taking lots of processing time which are not necessary for an AI agent to play.

The first section of this notebook looks at creating a barebones monopoly (obviously keeping the same functionality and gameplay) that can be run as quickly as possible.

The second section creates the base player class and then a human and AI player class that extend this base class. I identified 4 main decisions that are made during monopoly (excluding trading):


*   To buy a property (or start an auction for it)
*   A 'general' choice when it is the players turn. This general choice depends on what the player has:
 * If they own properties, they can mortgage any of them
 * If they have a colourset, they can buy a house on these properties (making sure to follow the house buying rules stated in the requirements)
 * If they have houses they can sell them
*   To bail out of jail if they are stuck there
* To make a big/small bid or leave an auction

There is one decision that I have not included, this is what properties to sell when paying off a large amount of rent. This is done automatically in the parent Player class.

I chose to automate this as it is very expensive to make a neural network for the AIs to choose what properties to sell and I felt that the impact of giving the AIs control over this would be minimal.

The third and final section is dedicated to making a training class. This class works by:  

1.   Making 6 AI agents with completely random genotypes
2.   Playing 75 games of monopoly with these agents, counting how many time each agent wins
3. Applying a crossover function and mutating the losers (the winner is the agent with the most wins out of those 75 games)

Completing these 3 steps constitutes 1 generation, and the class takes a parameter that lets the user choose how many generations to do (the more generation, the fitter the best individual, hopefully)

I am quite proud that I managed to get the training model to complete around **3 monopoly games a second** so each generation takes on average 25 seconds.

The genotype used in the main Unity project was trained over 250 generations, which equals **18750 monopoly games** and took around ~3hrs.



# Creating Monopoly environmnent

This first section creates classes for all the different types of tiles in the game, there is a very small parent class Tile that sets the position of each tile to the value given when it is instantiated. This is because, other than that, there is very little shared functionality between the other class

To improve the code readability, it would have been better to make an additional child of Tile class that represented buyable property as there are many shared functions and methods between stations, utilities and property.

Throughout these classes, there will be many commented out print statements, these allow anyone to manually test the game by giving the user a text interface to make decisions and see the game state. This was added to let anyone test the game and find any bugs or errors in the game.

These are commented out because the AI does not need a text interface and heavily slow down training.

Additionally, each class has its own landOn function that is called when a player lands on this class. Python does not handle overriding, so instead of making an abstract class in the parent tile class, each of the different tiles have their own method with the same name and input.

### Tile and Property Class

In [ ]:

#This is the parent tile class, all it does is set the position of the tile based on the input given.
#Self.auction is used to check whether an auction has been triggered by a
class Tile():
  def __init__(self,position):
    self.position = position
    self.auction = False




#This is the property class, contains a constructor that takes the: name, price, colour, position on board and an array of the different rent values
class Property(Tile):
  def __init__(self,name,price,colour,pos,rentArray):
    self.name = name
    self.price = price
    self.rent = rentArray[0]
    self.owner = "Available"
    self.mortgaged = False
    self.colour = colour
    self.rentArray = rentArray
    self.houses = 0
    self.house_price = 0

    # This determines the price of a house based on the colour
    if self.colour == ("Brown") or self.colour == ("Blue"):
      self.house_price = 50
    elif self.colour == ("Purple") or self.colour == ("Orange"):
      self.house_price = 100
    elif self.colour == ("Red") or self.colour == ("Yellow"):
      self.house_price = 150
    elif self.colour == ("Green") or self.colour == ("Deep Blue"):
      self.house_price = 200

    #This passes the position of the property to the parent constructor
    super().__init__(pos)

  #Function to buy the property
  def buy(self,player):
    if player.minusMoney(self.price):
      self.owner = player
      player.propertyColourArray.append(self.colour)
      player.properties.append(self)
      player.checkColourSet(self.colour)
    else:
      self.auction = True


  #Function to mortgage property
  def mortgage(self,player):
    if self.owner == player and self.houses == 0:
      self.mortgaged = True
      player.addMoney(self.price//2)


  def unmortgage(self,player):
    if self.owner == player:
      self.mortgaged = False
      player.minusMoney(self.price//2)
  #Function to sell property
  def sell(self,player):
    if self.owner == player:
      self.owner = "Available"
      player.addMoney(self.price)
      player.propertyColourArray.remove(self.colour)
      #Checks if the player had a colour set for this property and if so, remove that the player has that colour set
      if self.colour in player.colourset:
        player.colourset.remove(self.colour)

  #Function to charge rent to a player that is not the owner, minusMoneyRent is a special function that places the player in a state where it must sell properties until it can afford it
  def payrent(self, player):
    if self.owner != player:
      player.minusMoneyRENT(self.rent)
      self.owner.addMoney(self.rent)
      print("Charged Rent, amount: ", self.rent)


  #Function that runs when a player lands on the property, calling optionBuy if the property is free, charging rent if it is owned by another player, and nothing if the property is owned by that player or the property is mortgaged
  def landOn(self,player):
    if self.owner == "Available":
      self.optionBuy(player)
    elif self.owner != player:
      if not self.mortgaged:
        self.payrent(player)

  #Checks if the player has completed a circuit and then asks the AI player to make a decision. If it returns true, it buys the property
  def optionBuy(self,player):
    if player.circuit:
      if player.makeBuyDecision(self):
        self.buy(player)
      else:
        # This will start an auction, auction logic is held in the Board class
        self.auction = True

  #This increases rent by getting the corresponding value in rentArray
  def updateRent(self,player):
    if self.colour in player.colourset:
      print("Num of houses: ",self.houses)
      self.rent = self.rentArray[1+self.houses]

  #This will buy a house and run the updateRent function.
  #player.checkHouseVal will look if the player has a colour set and that the other properties in that set have equal house number
  def buyHouse(self,player):
    if (self.colour in player.colourset) and self.houses < 4:
      if player.checkHouseValid(self,1) and player.minusMoney(self.house_price):
        self.houses = self.houses + 1
        player.houseValues.append(self.house_price)
        self.updateRent(player)
      else:
        print("Unable to purchase")
        pass

  def sellHouse(self,player):
    if player.checkHouseValid(self,-1):
      self.houses = self.houses - 1
      player.addMoney(self.house_price)
      try:
        player.houseValues.remove(self.house_price)
      except:
        pass
      self.updateRent(player)
      return True
    else:
      return False




### Station class

In [ ]:
class Station(Tile):
  def __init__(self,name,price,pos,rentArray):
    self.name = name
    self.price = price
    self.rent = rentArray[0]
    self.owner = 'Available'
    self.mortgaged = False
    self.rentArray = rentArray
    self.colour = "NA"
    super().__init__(pos)


  def buy(self,player):
    if player.minusMoney(self.price):
      self.owner = player
      player.stations.append(self)
      self.updateRent(player)
    else:
      self.auction = True

  def mortgage(self,player):
    if self.owner == player:
      self.mortgaged = True
      player.addMoney(self.price//2)


  def unmortgage(self,player):
    if self.owner == player:
      self.mortgaged = False
      player.minusMoney(self.price//2)

  def sell(self,player):
    if self.owner == player:
      self.owner = "Available"
      player.addMoney(self.price)
      print("Selling station: ",self.name)
      self.updateRent(player)

  def payrent(self, player):
      player.minusMoneyRENT(self.rent)
      self.owner.addMoney(self.rent)
      print("Charged Rent, amount: ", self.rent)

  def landOn(self,player):
    if self.owner == "Available":
      self.optionBuy(player)
    elif self.owner != player:
      if not self.mortgaged:
        self.payrent(player)


  def optionBuy(self,player):
    if player.circuit:
      if player.makeBuyDecision(self):
        self.buy(player)
      else:
        self.auction = True
  #Rent is based on the number of stations the player owns, the value is clipped as sometimes when stations are sold,
  #length of stations can be 0 and then 0 - 1 = -1, which cannot be an index to an array
  def updateRent(self,player):
      self.rent = self.rentArray[np.clip(len(player.stations)-1,0,0)]



### Utilities class

In [ ]:
class Utilities(Tile):
  def __init__(self,name,price,pos,rentArray):
    self.name = name
    self.price = price
    self.rent = rentArray[0]
    self.owner = 'Available'
    self.mortgaged = False
    self.rentArray = rentArray
    self.colour = "NA"
    super().__init__(pos)


  def buy(self,player):
    if player.minusMoney(self.price):
      self.owner = player
      player.utils.append(self)
      self.updateRent(player)
    else:
      self.auction = True

  def mortgage(self,player):
    if self.owner == player:
      self.mortgaged = True
      player.addMoney(self.price//2)


  def unmortgage(self,player):
    if self.owner == player:
      self.mortgaged = False
      player.minusMoney(self.price//2)


  def sell(self,player):
    if self.owner == player:
      self.owner = "Available"
      player.addMoney(self.price)
      self.updateRent(player)


  def payrent(self, player):
      player.minusMoneyRENT(self.rent)
      self.owner.addMoney(self.rent)
      print("Charged Rent, amount: ", self.rent)

  def landOn(self,player):
    if self.owner == 'Available':
      self.optionBuy(player)
    elif self.owner != player:
      if not self.mortgaged:
        self.payrent(player)

  def optionBuy(self,player):
    if player.circuit:
      if player.makeBuyDecision(self):
        self.buy(player)
      else:
        self.auction = True


  def updateRent(self,player):
      self.rent = self.rentArray[np.clip(len(player.utils)-1,0,0)]



### Card Tiles

In [ ]:
class Card_space(Tile):
  def __init__(self,name,pos,cardArray):
    self.name = name
    self.cardArray = cardArray
    self.fineToCollect = 0
    super().__init__(pos)
  #Takes top card and then replaces at back of the queue
  def drawCard(self,player):
    nextCard = self.cardArray.pop(0)
    self.cardArray.append(nextCard)
    return nextCard


  def landOn(self,player):
    fine = self.playEffect(self.drawCard(player),player)
    if fine is not None:
      self.fineToCollect += fine


  #
  def playEffect(self,card,player):
    print("\n \nYou Got a: ",card.effect," card, with a value of ",card.amount, "or place of ",card.place)
    match card.effect:
      case "Get out of Jail":
        player.jail_cards += 1
      case "Give Money":
        player.addMoney(card.amount)
      case "Fine Money":
        player.minusMoney(card.amount)
        return card.amount
      case "Move Space No.":
        player.pos += card.amount
      case "Move Exact Space":
        player.pos = card.place
      case "Transfer Money To":
        card.collect(card.amount)
        player.addMoney(card.amount)
      case "Jail":
        player.GoToJail()



#A card has 3 values, a string called 'effect' that dictates what the card does, an 'amount',
#that represents how much of that effect (e.g. if the effect is 'Fine Money', amount represents how much to fine)
#and then place represents the position of where the player should be sent if the effect is "Move Exact Space"
#If amount or place are not relevant to a card's effect, they are set to 0
class Card():
  def __init__(self,effect,amount,place):
    self.effect = effect
    self.amount = amount
    self.place =place


### Free parking, Go to Jail and Tax

In [ ]:
#addAmount is called whenever a player is subject to a fine (the cases this occurs is seen in functional requirements document)
#When a player lands on it, they get all the money on free parking and the amount is set to 0
class Free_Parking(Tile):
  def __init__(self,pos,name):
    self.amount = 0
    self.name = name
    super().__init__(pos)

  def addAmount(self,amount):
    self.amount += amount

  def landOn(self,player):
    player.addMoney(self.amount)
    self.amount = 0

#If the player lands here, they are sent to jail
class Go_jail(Tile):
  def __init__(self,pos,name):
    self.name = name
    super().__init__(pos)
  def landOn(self,player):
    player.GoToJail()

#If the player lands here, they lose the money stated in the amount field
class Tax(Tile):
  def __init__(self,name,pos,amount):
    self.name = name
    self.amount = amount
    super().__init__(pos)

  def landOn(self,player):
    player.minusMoney(self.amount)

### Jail and Go

In [ ]:
#If they land on the tile Jail, they are 'just visiting' and should not be affected by anything
class Jail(Tile):
  def __init__(self,name,pos):
    self.name = name
    self.prisoners = []
    super().__init__(pos)

  def landOn(self,player):
    pass


class Go(Tile):
  def __init__(self,name):
    self.name = name
    super().__init__(0)
  #Giving money is done in the Board class
  def landOn(self,player):
    pass

### Board Class

In [ ]:

import random as rand
class Board():
  def __init__(self,*args,**kwargs):
    self.Tiles = []
    self.players = [arg for arg in args]
    rand.shuffle(self.players)
    self.round = 0
    self.turnPointer = 0
    self.winner = None
    self.InitialiseBoard()

  #Rolls 2 random integers from 1 to 6 and adds them, returns 2 values, the first is the sum of the dice,
  #the second is whether it was a double or not
  def rollDice(self):
    dice1 = rand.randint(1,6)
    dice2 = rand.randint(1,6)
    print("\n \n Dice 1: ",dice1)
    print("Dice 2: ",dice2,"\n \n")

    if dice1 == dice2:
      return dice1+dice2,True
    else:
      return dice1+dice2,False

  #for these add tile functions, position is set to the end of the current list of tiles.
  #The board initialises the tiles in order of their in-game position
  def addProperty(self,name,price,colour,rentArray):
    pos = len(self.Tiles)
    newProperty = Property(name,price,colour,pos,rentArray)
    self.Tiles.append(newProperty)

  def addStation(self,name,price,rentArray):
    pos = len(self.Tiles)
    newStation = Station(name,price,pos,rentArray)
    self.Tiles.append(newStation)

  def addUtility(self,name,price,rentArray):
    pos = len(self.Tiles)
    newUtility = Utilities(name,price,pos,rentArray)
    self.Tiles.append(newUtility)

  def addCard_space(self,name,cardArray):
    pos = len(self.Tiles)
    newCardSpace = Card_space(name,pos,cardArray)
    self.Tiles.append(newCardSpace)

  def addFree_parking(self):
    pos = len(self.Tiles)
    newFreeParking = Free_Parking(pos,"Free Parking")
    self.Tiles.append(newFreeParking)

  def addGoToJail(self):
    pos = len(self.Tiles)
    newGoToJail = Go_jail(pos,"Go to Jail")
    self.Tiles.append(newGoToJail)

  def addTax(self,name,amount):
    pos = len(self.Tiles)
    newTax = Tax(name,pos,amount)
    self.Tiles.append(newTax)

  def addJail(self):
    pos = len(self.Tiles)
    newJail = Jail("Jail",pos)
    self.Tiles.append(newJail)

  def addGo(self):
    newGo = Go("Go")
    self.Tiles.append(newGo)

  #This creates the base set of cards, removing the description and simplifying them to just the effect they have
  def createOpportunityKnocks(self):
    OKCard = [Card("Give Money",50,0),
              Card("Give Money",100,0),
              Card("Move Exact Space",0,39),
              Card("Move Exact Space",0,24),
              Card("Fine Money",15,0),
              Card("Fine Money",150,0),
              Card("Move Exact Space",0,15),
              Card("Give Money",150,0),
              Card("Fine Money",200,0),
              Card("Move Exact Space",0,0),
              Card("Fine Money",150,0),
              Card("Move Space No.",-3,0),
              Card("Moce Exact Space",0,11),
              Card("Jail",0,0),
              Card("Fine Money",30,0),
              Card("Get out of Jail",0,0)]
    #Shuffle order of cards
    rand.shuffle(OKCard)
    return OKCard

  def createPotLuck(self):
    PLCard = [Card("Give Money",200,0),
              Card("Give Money",50,0),
              Card("Move Exact Space",0,1),
              Card("Give Money",20,0),
              Card("Give Money",200,0),
              Card("Fine Money",100,0),
              Card("Fine Money",50,0),
              Card("Move Exact Space",0,0),
              Card("Give Money",50,0),
              Card("Fine Money",10,0),
              Card("Fine Money",50,0),
              Card("Give Money",100,0),
              Card("Jail",0,0),
              Card("Give Money",25,0),
              Card("Give Money",40,0),
              Card("Get out of Jail",0,0)]
    rand.shuffle(PLCard)
    return PLCard

  #Gets the index based on a given name
  def FindTileByName(self,name):

    tilePos = 0
    for i in range(len(self.Tiles)):
      if self.Tiles[i].name == name:
        tilePos = i
    return tilePos

  # This is where the board constructs itself with all of the tiles and giving the card spaces their shuffled cards
  def InitialiseBoard(self):
    potluckcards = self.createPotLuck()
    OKCards = self.createOpportunityKnocks()
    self.addGo()
    self.addProperty("The Old Creek",60,"Brown",[2,4,10,30,90,160,250])
    self.addCard_space("Pot Luck",potluckcards)
    self.addProperty("Gangster Paradise",60,"Brown",[4,8,20,60,180,320,450])
    self.addTax("Tax",200)
    self.addStation("Brighton Station",200,[25,50,100,200])
    self.addProperty("The Angels Delight",100,"Blue",[6,12,30,90,270,400,550])
    self.addCard_space("Opportunity Knocks",OKCards)
    self.addProperty("Potter Avenue",100,"Blue",[6,12,30,90,270,400,550])
    self.addProperty("Granger Drive",120,"Blue",[8,16,40,100,300,450,600])
    self.addJail()
    self.addProperty("Skywalker Drive",140,"Purple",[10,20,50,150,450,625,750])
    self.addUtility("Tesla Power Co",150,[28,70])
    self.addProperty("Wookie Hole",140,"Purple",[10,20,50,150,450,625,750])
    self.addProperty("Rey Lane",160,"Purple",[12,24,60,180,500,700,900])
    self.addStation("Hove Station",200,[25,50,100,200])
    self.addProperty("Bishop Drive",180,"Orange",[14,28,70,200,550,750,950])
    self.addCard_space("Pot Luck",potluckcards)
    self.addProperty("Dunham Street",180,"Orange",[14,28,70,200,550,750,950])
    self.addProperty("Broyles Lane",200,"Orange",[16,32,80,220,600,800,1000])
    self.addFree_parking()
    self.addProperty("Yue Fei Square",220,"Red",[18,36,90,250,700,875,1050])
    self.addCard_space("Opportunity Knocks",OKCards)
    self.addProperty("Mulan Rouge",220,"Red",[18,36,90,250,700,875,1050])
    self.addProperty("Han Xin Gardens",240,"Red",[20,40,100,300,750,925,1100])
    self.addStation("Falmer Station",200,[25,50,100,200])
    self.addProperty("Shatner Close",260,"Yellow",[22,44,110,330,800,975,1150])
    self.addProperty("Picard Avenue",260,"Yellow",[22,44,110,330,800,975,1150])
    self.addUtility("Edison Water",150,[28,70])
    self.addProperty("Crusher Creek",280,"Yellow",[22,44,120,360,850,1025,1200])
    self.addGoToJail()
    self.addProperty("Sirat Mews",300,"Green",[26,52,130,390,900,1100,1275])
    self.addProperty("Ghengis Crescent",300,"Green",[26,52,130,390,900,1100,1275])
    self.addCard_space("Pot Luck",potluckcards)
    self.addProperty("Ibis Close",320,"Green",[28,56,150,450,1000,1200,1400])
    self.addStation("Portslade Station",200,[25,50,100,200])
    self.addCard_space("Opportunity Knocks", OKCards)
    self.addProperty("James Webb Way",350,"Deep Blue",[35,70,175,500,1100,1300,1500])
    self.addTax("Super Tax",100)
    self.addProperty("Turing Heights",400,"Deep Blue",[50,100,200,600,1400,1700,2000])


  #Call this to see that all tiles are correctly on the board
  def checkBoard(self):
    for Tile in self.Tiles:
      print("Name: %r  Position: %r" % (Tile.name,Tile.position))

  # If the game goes on for too long, it automatically ends and calculates the asset values of each player
  def calculateAssets(self,player):
    price_array = np.transpose([[Property.price for Property in player.properties],[i for i in range(len(player.properties))]])


      #Checks all player assets to see if the player can even pay this rent
    total_assets = player.money + np.sum(price_array[:,0]) + 200*len(player.stations) + 150*len(player.utils) + np.sum(player.houseValues)
    return total_assets

  def TakeTurn(self):

    if len(self.players) < 2:
      print("\n \n Congratulations!!!! The winner of Monopoly is!: ",self.players[0].name)
      self.winner = self.players[0]
      return "Game Over"
    elif self.players[0].turn > 1000:
      print("\n \nThe game is taking too long, leaving early and player with most assets wins")
      assetlist = []
      for i in range(len(self.players)):
        assetlist.append([self.calculateAssets(self.players[i]),i])
      assetlist = np.array(assetlist)
      sortedasset = assetlist[assetlist[:,0].argsort()]
      self.winner = self.players[int(sortedasset[-1][1])]
      print("\n\n Congratulation!! The winner is: ",self.winner.name)
      return "Game Over"

    #Checks if the previous turn was the last player in the list, if so, go back to 0

    if self.turnPointer >= len(self.players):
      self.turnPointer = 0

    #Grab next player
    currentPlayer = self.players[self.turnPointer]
    currentPlayer.turn += 1
    playerProperties = [prop.name for prop in currentPlayer.properties]
    playerStations = [stat.name for stat in currentPlayer.stations]
    print("Current Player's turn: ", currentPlayer.name, "\n Players properties: ", playerProperties, "\n Player's money: ",currentPlayer.money,"\n Player's Stations: ",playerStations,"\n Player's Utils: ",len(currentPlayer.utils))

    #This checks if the player is in jail, if they are, check if they have served 2 turns,
    #if not, give them the choice to bail or stay another turn, either way they lose their turn
    if currentPlayer.inJail:
      if currentPlayer.jailTurn == 2:
        currentPlayer.LeaveJail()
        self.turnPointer += 1
        return None
        #makeJailChoice returns True if the player wishes to leave jail (either paying 50 or using a get out of jail free card)
      elif currentPlayer.makeJailChoice():
        freeParking = self.Tiles[20]
        freeParking.addAmount(50)
        currentPlayer.LeaveJail()
        self.turnPointer += 1
        return None
      else:
        currentPlayer.jailTurn += 1
        self.turnPointer += 1
        return None

    #Rolls the dice and returns the value of the roll and whether it was a double
    diceRoll,isDouble = self.rollDice()
    print("Rolled a :", diceRoll)

    #Checks if the player has rolled 3 doubles in a row this turn
    if isDouble:
      currentPlayer.doubleCount += 1
      #If so, send to jail and end turn
      if currentPlayer.doubleCount >= 3:
        currentPlayer.doubleCount = 0
        isDouble = False
        currentPlayer.GoToJail()
        self.turnPointer += 1
        return None

    #Moves the player based on the value of their roll
    self.movePlayer(currentPlayer,diceRoll)

    if currentPlayer.dead:
      self.players.remove(currentPlayer)
      return None
    #EndTurn is a boolean that represents whether the player has chosen to end turn
    currentPlayer.endTurn = False

    #Lets the player make a 'general' choice, if they have properties, they can choose to mortgage them or build houses if they own a set etc.
    while not currentPlayer.endTurn:
      currentPlayer.makeGeneralChoice()

    #If the player rolled a double, then they take another turn, this is done with a recursive call of this method, as the turn pointer has not changed,
    #It should be the same player playing again
    if isDouble:
      self.turnPointer -= 1
    #If the player did not roll a double this turn, set double counter back to 0 and then increment turnPointer so next player can play
    else:
      currentPlayer.doubleCount = 0
    self.turnPointer += 1

  def movePlayer(self,player,diceval):
    new_pos = player.pos + diceval
    #Checks if made a loop around board, if so, give 200
    if new_pos >= len(self.Tiles):
      new_pos = new_pos - (len(self.Tiles))

      if not player.circuit:
        player.circuit = True

      player.addMoney(200)
    #moves player to new tile and calls the landOn function at that tile

    Tile = self.Tiles[new_pos]
    player.pos = Tile.position
    print("Player landed on: ",Tile.name,"with id ",player.pos)
    Tile.landOn(player)

    freeParking = self.Tiles[20]

    #This if else block will check whether the tile the player is on requires any board management, like starting an auction or increasing freeParking
    if Tile.auction:
      Tile.auction = False
      self.auctionMode(Tile)
    elif Tile.name == "Opportunity Knocks" or Tile.name == "Pot Luck":
      freeParking.addAmount(Tile.fineToCollect)
      Tile.fineToCollect = 0
    elif Tile.name == "Tax":
      freeParking.addAmount(Tile.amount)
    elif Tile.name == "Super Tax":
      freeParking.addAmount(Tile.amount)




  def auctionMode(self,Tile):
    #Makes a list with all players that have completed a circuit
    auctionPlayers = [player for player in self.players if player.circuit]
    auctionPrice = Tile.price/4

    #Check if there are enough players to do auction
    if len(auctionPlayers) < 2:
      #print("Not enough players to complete an auction")
      pass

    #Continue doing auction until only one player remains, gives the player the option to do a big bid or regular bid
    else:
      onGoing = True
      i = 0
      while onGoing:
        if i >= len(auctionPlayers):
          i = 0
        print([player.name for player in auctionPlayers])
        print("Number of Bidders: ",len(auctionPlayers))
        print("\n \n Current Bidder: ", auctionPlayers[i].name,"\n \n")
        response = auctionPlayers[i].bid(auctionPrice,Tile)
        if response == "Bid":
          auctionPrice += 25
          i += 1
        elif response == "Big Bid":
          auctionPrice += 200
          i += 1
        else:
          auctionPlayers.remove(auctionPlayers[i])
          if len(auctionPlayers) <= 1:
            onGoing = False
          i += 1



      #The winner should be the only player in the list, then take the money out and make them the owner
      winner = auctionPlayers[0]
      print("The auction winner is!: ",winner.name)
      costdiff = Tile.price - auctionPrice
      print("You bought ",Tile.name, "for the price of ",auctionPrice," which is a cost difference of ", costdiff, " from the original price")
      winner.addMoney(costdiff)
      Tile.buy(winner)

    #This is a debug function to test renting and end game, lets any player buy every property
  def buyAllProp(self,player):
    properties = [tile for tile in self.Tiles if isinstance(tile,Property)]
    player.money = 99999
    for prop in properties:
      prop.buy(player)
    for prop in properties:
      prop.buyHouse(player)
    player.money = 100

  def buyAllStatandUtil(self,player):
    stations = [tile for tile in self.Tiles if (isinstance(tile,Station) or isinstance(tile,Utilities))]
    player.money = 950
    for station in stations:
      station.buy(player)




## Test that Board has initialised correctly

Run the code segment below to view the position of all the tiles on the board, the last line shows the cards in opportunity knocks, this is randomised, so every time this is run, a different order should appear

In [ ]:
board = Board()
board.checkBoard()
print([card.effect for card in board.Tiles[7].cardArray])

Name: 'Go'  Position: 0
Name: 'The Old Creek'  Position: 1
Name: 'Pot Luck'  Position: 2
Name: 'Gangster Paradise'  Position: 3
Name: 'Tax'  Position: 4
Name: 'Brighton Station'  Position: 5
Name: 'The Angels Delight'  Position: 6
Name: 'Opportunity Knocks'  Position: 7
Name: 'Potter Avenue'  Position: 8
Name: 'Granger Drive'  Position: 9
Name: 'Jail'  Position: 10
Name: 'Skywalker Drive'  Position: 11
Name: 'Tesla Power Co'  Position: 12
Name: 'Wookie Hole'  Position: 13
Name: 'Rey Lane'  Position: 14
Name: 'Hove Station'  Position: 15
Name: 'Bishop Drive'  Position: 16
Name: 'Pot Luck'  Position: 17
Name: 'Dunham Street'  Position: 18
Name: 'Broyles Lane'  Position: 19
Name: 'Free Parking'  Position: 20
Name: 'Yue Fei Square'  Position: 21
Name: 'Opportunity Knocks'  Position: 22
Name: 'Mulan Rouge'  Position: 23
Name: 'Han Xin Gardens'  Position: 24
Name: 'Falmer Station'  Position: 25
Name: 'Shatner Close'  Position: 26
Name: 'Picard Avenue'  Position: 27
Name: 'Edison Water'  Posi

# Defining a parent Player Class

As mentioned above, there are 4 main decisions players make in this monopoly simulator, the code for these decisions are in  

1.   Human player class for the text interface version
2.   AI class for the neural network version.

In the parent class, there is the automatic selling function, the functions for detecting whether a player has a colourset, and if they choose to buy a house, a function to check whether they can buy it on a certain property.

In [ ]:
import numpy as np


class Player():
  def __init__(self,name):
    self.name = name
    self.money = 500
    self.circuit = False
    self.propertyColourArray = []
    self.colourset = []
    self.properties = []
    self.stations = []
    self.utils = []
    self.houseValues = []
    self.jail_cards = 0
    self.pos = 0
    self.inJail = False
    self.jailTurn = 0
    self.doubleCount = 0
    self.endTurn = False
    self.dead = False
    self.turn = 0

  def addMoney(self,cost):
    self.money += cost
    self.money = np.clip(self.money,0,2500)

  def minusMoney(self,cost):
    if (self.money-cost) >= 0:
      self.money -= cost
      return True
    else:
      return False

  def minusMoneyRENT(self,cost):
    if (self.money-cost) >= 0:
      self.money -= cost
    else:
      #The first line creates a matrix with each row having the price and then index of that property, in theory ordering them by price with their index so they can be retrieved
      #Second line sorts the matrix by the first column (price)
      price_array = np.transpose([[Property.price for Property in self.properties],[i for i in range(len(self.properties))]])
      price_array_sorted = price_array[price_array[:,0].argsort()]


      #Checks all player assets to see if the player can even pay this rent
      total_assets = self.money + np.sum(price_array[:,0]) + 200*len(self.stations) + 150*len(self.utils) + np.sum(self.houseValues)
      print("Total Assets ",total_assets)
      print("Cost to pay ",cost)
      if total_assets >= cost:
        #This is an auto sell function, instead of letting a player or AI make the decision of what properties to sell here, it instead sorts the selling order by:
        # 1. Sell utilities if there are any
        # 2. Sell stations if there are any
        # 3. Sell cheapest property that has no house
        # 4. If all properties have houses, sell house of cheapest property, given that it is valid to sell
        bias = 1
        while (self.money-cost) < 0 and len(self.properties) > 0:

          if len(self.utils) > 0:
            next_util = self.utils.pop()
            next_util.sell(self)
            print("Sold a Util: ",next_util.name)

          elif len(self.stations) > 0:
            next_station = self.stations.pop()
            next_station.sell(self)
            print("Sold a station: ",next_station.name)

          elif len(price_array_sorted) > 0:
            yet_to_sell = True
            i = 0

            while yet_to_sell:

              if len(self.properties) <= 0:
                self.dead = True
                return None
              elif (self.properties[np.clip(price_array_sorted[i][1],0,len(self.properties)-1)]).houses <= 0:
                cheapest_prop =self.properties.pop(np.clip(price_array_sorted[i][1],0,len(self.properties)-1))
                cheapest_prop.sell(self)
                bias += 1
                print("Sold a property: ", cheapest_prop.name)
                yet_to_sell = False

              elif i < len(price_array_sorted)-bias:
                i += 1

              else:
                i = 0
                j = 0
                yet_to_sell_house = True

                while yet_to_sell_house:

                  cheapest_prop = self.properties[np.clip(price_array_sorted[j][1],0,len(self.properties)-1)]
                  if cheapest_prop.houses <= 0:
                    return None
                  elif cheapest_prop.sellHouse(self):
                    yet_to_sell_house = False
                  else:
                    j += 1
        self.money -= cost
        print("Well Done, you payed off your rent")
      else:
        #If the player does not have enough money, they die
        self.dead = True
        print("You did not pay your rent, you have lost")

  def checkColourSet(self,colour):
    props = [prop for prop in self.properties if prop.colour == colour]
    num_of_colour = [prop.colour for prop in props]
    if colour == "Deep Blue" or colour == "Brown":
      if len(num_of_colour) == 2:
        self.colourset.append(colour)
        for prop in props:
          prop.updateRent(self)
    else:
      if len(num_of_colour) == 3:
        self.colourset.append(colour)
        for prop in props:
          prop.updateRent(self)

  def checkHouseValid(self,tile,val):
    matching_colour_house_num = [prop.houses for prop in self.properties if prop.colour == tile.colour]
    # np.ptp checks the range of values in an array, so if it returns 0, then all tiles have the same number of houses, in this case it can buy a house
    if tile.houses < 0:
      return False
    if np.ptp(matching_colour_house_num) == 0:
      return True
    # Otherwise, check if incrementing or decrementing the number of houses on a tile exceeds the current max or min number of houses. If it is in this range after adjustment
    # It can safely sell
    elif (tile.houses+val <= np.max(matching_colour_house_num)) and (tile.houses+val >= np.min(matching_colour_house_num)):
      return True
    else:
      return False

  def GoToJail(self):
    self.pos = 10
    self.inJail = True

  def LeaveJail(self):
    self.inJail = False

## Creating a human player class to test board manually

The reason I made a human player class was to test the monopoly environment and make sure everything works as intended.

It implements the 4 decisions mentioned above by giving a text prompt and letting the person enter an input. After testing it manually (and using some of the debug methods I made in Board), I concluded that this game met all functional requirements for the full game (excluding giving the player control of selling when paying rent).

In [ ]:
class HumanPlayer(Player):
  def __init__(self,name):
    super().__init__(name)

  def makeBuyDecision(self,tile):

    answer = input("""\n Would you like to buy: {0} \n \n The price for this property is: {1}  \n \n
    If so, please enter 'Y' to say yes or anything else for no\n""".format(tile.name,tile.price))

    if answer.upper() == "Y":
      return True
    else:
      return False

  def makeJailChoice(self):
    if self.jail_cards > 0:
      answer = input("Would you like to use a jail card? \n If so, please enter 'Y' to say yes or anything else for no\n")

      if answer.upper() == "Y":
        return True
      else:
        return False
    elif self.money >= 50:
      answer = input("Would you like to pay a £50 bail? \n If so, please enter 'Y' to say yes or anything else for no \n")
      if answer.upper() == "Y":
        self.money -= 50
        return True
      else:
        return False
    else:
      return False

  def makeGeneralChoice(self):
    if len(self.properties) > 0:
      answer = input("Would you like to mortgage a property? \n If so, please enter 'Y' to say yes or anything else for no\n")
      if answer.upper() == "Y":
        for i in range(len(self.properties)):
          if self.properties[i].houses == 0:
            print(i,": ",self.properties[i].name)

        property_num = int(input("Please enter than number for the property you would like to mortgage: \n"))
        propertyToMortgage = self.properties[property_num]
        propertyToMortgage.mortgage(self)

      if len(self.colourset) > 0:
        answer_house = input("Would you like to buy a house? \n If so, please enter 'Y' to say yes or anything else for no\n")
        if answer_house.upper() == "Y":

          for i in range(len(self.properties)):
            if self.properties[i].colour in self.colourset:
              print(i,": ",self.properties[i].name," Number of Houses: ",self.properties[i].houses)
          house_num = int(input("Please enter than number for the property you would like to buy a house for: \n"))
          self.properties[house_num].buyHouse(self)
        list_prop_with_house = [prop for prop in self.properties if prop.houses > 0]

        if len(list_prop_with_house) > 0:
          answer_house_sell = input("Would you like to sell a house? \n If so, please enter 'Y' to say yes or anything else for no\n")
          if answer_house_sell.upper() == "Y":
            for i in range(len(list_prop_with_house)):
              print(i,": ", list_prop_with_house[i].name," Number of Houses: ",list_prop_with_house[i].houses)
            house_num = int(input("Please enter than number for the property you would like to sell a house for: \n"))
            list_prop_with_house[house_num].sellHouse(self)

    answer_end_turn = input("Would you like to end your turn? \n If so, please enter 'Y' to say yes or anything else for no\n")
    if answer_end_turn.upper() == "Y":
      self.endTurn = True

  def bid(self,amount,Tile):
    if amount+200 <= self.money:
      print("Current bid is: ",amount)
      print("1) Bid: ",amount+25)
      print("2) Big Bid: ", amount+200)
      print("3) Leave")
      answer = input("Please enter either 'Bid' or 'Big Bid' to make a bid or anything else to leave\n")
    elif amount+25 <= self.money:
      print("Current bid is: ",amount)
      print("You can only afford to do a regular bid")
      print("1) Bid: ",amount+25)
      print("2) Leave")
      answer = input("Please enter either 'Bid' to make a bid or anything else to leave\n")
    else:
      print("You cannot afford to bid, you are kicked out of the auction")
      answer = None
    return answer






# Playing the game with human players

~As mentioned above to play the game with people **please uncomment all print statements in the above code and in the code snippet below**, otherwise there will be no text interface to play the game.~ This is the human version, I would **NOT** recommend running the AI agents on this version, it will severely lag the notebook as it will be printing immense amount of text quickly, this notebook is to test the monopoly environment

It may seem that at first no one can buy property, but that is because everyone must complete a circuit of the board first to buy. The text interface is quite specific, so make sure to not misspell an input.

Also, the player's money is the amount ***before*** rolling, so any fines or properties purchased will not change the value shown of the player's money (internally it is updated of course), until the next turn.

To add and remove players, first create a new HumanPlayer object, giving a string as a name, then make sure the Board object takes these players as inputs.

The while statement makes sure that the game keeps going until it returns "Game Over"

***For the sake of convenience, there will be another notebook in the same folder as this one titled 'Human Simulated Monopoly', that is identical to this notebook, but ready for people to play.***

In [ ]:
player1 = HumanPlayer("Alex")
player2 = HumanPlayer("Evie")
player3 = HumanPlayer("Jude")
player4 = HumanPlayer("Rudy")
player5 = HumanPlayer("Rohan")
#Order of the players is shuffled each game
board = Board(player1,player2,player3,player4,player5)

while board.TakeTurn() != "Game Over":
  pass

Current Player's turn:  Alex 
 Players properties:  [] 
 Player's money:  500 
 Player's Stations:  [] 
 Player's Utils:  0

 
 Dice 1:  4
Dice 2:  1 
 

Rolled a : 5
Player landed on:  Brighton Station with id  5


KeyboardInterrupt: Interrupted by user

## Creating a base AI agent


Here is where the AI agent is implemented. It uses the pyTorch library to create weight tensors that represent the weights of different layers of the network. The activation function used is the sigmoid function, I did not have the time to experiment with many different activation function, so instead I opted for what I was familiar with.

The AI is based on a genetic algorithm approach, which is an evolutionary technique focused on slowly improving a genotype with random mutation and crossovers to reach an optimum solution.

This notebook will not go into detail about genetic algorithms, if you are interested in learning the fundamental elements, I recommend *An Introduction to Genetic Algorithm*s, by Melanie Mitchell.

However, it will explain how this algorithm implements the:


*   Genotype to phenotype mapping
*   Mutation function
*   Fitness function
*   Crossover function

**Firstly**, the genotype to phenotype mapping. The genotype for the agent has 20 genes. Each gene is a pyTorch tensor of varying sizes (this size depending on how large the layer it represents is) that is initialised to random weights. Each layer has both a weight tensor and a bias tensor. The genotype consists of 10 weight tensors and 10 bias tensors, meaning there are 10 distinct layers in this agent. There are 4 separate networks, 1 for each decision with each network having different number of layers and size of layers. Below is a graphical depiction of each network.

Make Buy decision: 3 network layers (1 input layer) [13->5->3->1]
nn.svg

Make Jail Choice: 2 network layers [4->2->1]
nn (1).svg

Make General Choice: 3 network layers [13->10->6->6]
nn (2).svg

Bid: 2 network layers [14->4->1]
nn (3).svg

(Graphs generated in https://alexlenail.me/NN-SVG/index.html)



For most of the network layers, there is a single output, this output is a value from 0-1, a value greater than 0.5 is regarded as a 'yes' to the decision (except for bidding and general choice, details for those explained below)

**Secondly,** the mutation function. Due to the already relatively large size of the networks, I opted for a very simple mutation, the creep mutation. This adds a gaussian noise (mean at 0, SD based on mutation rate) to each layer of the network, if the agent does not win.

**Thirdly,** the fitness function. As monopoly is still immensely based on chance, to give a reliable fitness function, it must first simulate a large quantity of games. This fitness function simply counts how many times each agent wins, the agent with the most wins is deemd the winner of that generation.

**Finally,** the crossover function. Once again, as each agent is quite big, the crossover function is very simple. Typically, a crossover function would look at each individual parameter, decide a crossover point, and then merge the two genotypes together. However in this case, the crossover function looks at each gene in the genotype (layer weights) and gives it a chance to be replaced with the winners weights.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random



class AI_Player(Player):
  def __init__(self,id,genotype):
    #The genotype should be a list of torch tensors, where each tensor represents the weights and biases for a network layer
    #1-3 is for buy decision
    self.W_1 = genotype[0]
    self.b_1 = genotype[1]
    self.W_2 = genotype[2]
    self.b_2 = genotype[3]
    self.W_3 = genotype[4]
    self.b_3 = genotype[5]

    #4-5 is for jail decision
    self.W_4 = genotype[6]
    self.b_4 = genotype[7]
    self.W_5 = genotype[8]
    self.b_5 = genotype[9]

    #6-8 is for general decision:
    self.W_6 = genotype[10]
    self.b_6 = genotype[11]
    self.W_7 = genotype[12]
    self.b_7 = genotype[13]
    self.W_8 = genotype[14]
    self.b_8 = genotype[15]
    self.sigmoid = nn.Sigmoid()


    #9-10 is for bidding:
    self.W_9 = genotype[16]
    self.b_9 = genotype[17]
    self.W_10 = genotype[18]
    self.b_10 = genotype[19]

    super().__init__(id)


  def makeBuyDecision(self,tile):
    input_money = [(self.money-1500)/1500,-tile.price/200]
    input_tile_colour = [self.colourToNumber(tile.colour)/7]
    input_colours = [self.countColours("Brown")/2,
                     self.countColours("Blue")/3,
                     self.countColours("Purple")/3,
                     self.countColours("Orange")/3,
                     self.countColours("Red")/3,
                     self.countColours("Yellow")/3,
                     self.countColours("Green")/3,
                     self.countColours("Deep Blue")/2]
    input_stations = [len(self.stations)]
    input_utils = [len(self.utils)]
    input_p_turns = [self.turn]
    all_input = input_money + input_tile_colour +input_colours + input_stations + input_utils + input_p_turns

    final_input = torch.tensor([[float(input) for input in all_input]])
    #Calculating output

    h1 = self.sigmoid(torch.mm(final_input,self.W_1)+self.b_1)
    h2 = self.sigmoid(torch.mm(h1,self.W_2)+ self.b_2)
    output = self.sigmoid(torch.mm(h2,self.W_3)+ self.b_3)
    if output < 0.5:
      return False
    else:
      return True

  def countColours(self,colour):
    counter = 0
    for col in self.propertyColourArray:
      if col == colour:
        counter += 1
    return counter

  def colourToNumber(self,colour):
    match colour:
      case "NA":
        return 0
      case "Brown":
        return 0
      case "Blue":
        return 1
      case "Purple":
        return 2
      case "Orange":
        return 3
      case "Red":
        return 4
      case "Yellow":
        return 5
      case "Green":
        return 6
      case "Deep Blue":
        return 7

  def makeJailChoice(self):
    #For the sake of simplicity, the agent will only care about: the number of get out of jail free cards, money
    #and turns spent in jail as the input

    final_input = torch.tensor([[float(self.jail_cards),float(self.money),float(self.jailTurn),float(self.turn)]])



    h1 = self.sigmoid(torch.mm(final_input,self.W_4)+self.b_4)
    output = self.sigmoid(torch.mm(h1,self.W_5)+self.b_5)
    if output < 0.5:
      return False
    else:
      return True


  def makeGeneralChoice(self,):
    #For the sake of simplicity, the AI only has one round of choice, it automatically chooses to end turn after its actions

    input_money = [(self.money-1500)/1500]
    input_colours = [self.countColours("Brown")/2,
                     self.countColours("Blue")/3,
                     self.countColours("Purple")/3,
                     self.countColours("Orange")/3,
                     self.countColours("Red")/3,
                     self.countColours("Yellow")/3,
                     self.countColours("Green")/3,
                     self.countColours("Deep Blue")/2]
    input_stations = [len(self.stations)]
    input_utils = [len(self.utils)]
    input_sum_house_val = [np.sum(self.houseValues)]
    input_p_turn = [self.turn]
    all_input = input_money + input_colours + input_stations + input_utils + input_sum_house_val + input_p_turn

    final_input = torch.tensor([[float(input) for input in all_input]])




    h1 = self.sigmoid(torch.mm(final_input,self.W_6)+self.b_6)
    h2 = self.sigmoid(torch.mm(h1,self.W_7)+self.b_7)
    #output will be a vector with 6 components:
    # 1. Does it want to mortgage
    # 2. What property to mortgage if yes
    # 3. Does it want to buy a house?
    # 4. For what property?
    # 5. Does it want to sell a house?
    # 6. For what property?
    output = self.sigmoid(torch.mm(h2,self.W_8)+self.b_8)
    if (len(self.properties) > 0):
      #Finds the appropriate property by multiplying the decimal by the number of properties
      # e.g. if output[1] was 0.5 and there were 4 properties, then the formula would give 1, which would be the second property
      # i.e halfway through the list


      #if output[0][0] > 0.5:
       # propertymortgageid = int(np.round(output[0][1] * len(self.properties))-1)
        #self.properties[propertymortgageid].mortgage(self)
      if len(self.colourset) > 0 and output[0][2] > 0.5:
        propertyhousearray = [tile for tile in self.properties if tile.colour in self.colourset]
        if len(propertyhousearray) > 0:
          propertyhouseid = int(np.round(random.random()*len(propertyhousearray))-1)
          #print("Bought a house on: ",propertyhousearray[propertyhouseid].name)
          propertyhousearray[propertyhouseid].buyHouse(self)


      #if len(self.houseValues) > 0 and output[0][4] > 0.5:
       # propertysellarray = [tile for tile in self.properties if tile.houses > 0]
       # propertysellid = int(np.round(output[0][5]*len(propertysellarray))-1)
        #propertysellarray[propertysellid].sellHouse(self)


    self.endTurn = True

  def bid(self,amount,tile):
    #The bidding process will be really slow, as each time the AIs will have to have a whole new wave of inputs and go through all the calculations

    #Instead of letting the AI bid 25 if it cannot afford to big bid, it will just leave as soon as it cannot even pay big bid.
    if amount+200 > self.money:
      return None

    input_money = [(self.money-1500)/1500,-tile.price/200]
    input_current_bid = [amount]
    input_tile_colour = [self.colourToNumber(tile.colour)/7]
    input_colours = [self.countColours("Brown")/2,
                     self.countColours("Blue")/3,
                     self.countColours("Purple")/3,
                     self.countColours("Orange")/3,
                     self.countColours("Red")/3,
                     self.countColours("Yellow")/3,
                     self.countColours("Green")/3,
                     self.countColours("Deep Blue")/2]
    input_stations = [len(self.stations)]
    input_utils = [len(self.utils)]
    all_input = input_money + input_current_bid + input_tile_colour +input_colours + input_stations + input_utils

    final_input = torch.tensor([[float(input) for input in all_input]])
    #Calculating output

    h1 = self.sigmoid(torch.mm(final_input,self.W_9)+self.b_9)
    output = self.sigmoid(torch.mm(h1,self.W_10)+ self.b_10)

    if output < 0.33:
      return "Bid"
    elif output < 0.66:
      return "Big Bid"
    else:
      return None

## Testing environment with 6 AI bots

This demonstrates what a generation of monopoly games might look like. After it is completed, the scoreboard for those games is printed. As these genotypes are randomly generated, none should do better than the other, hence the scoreboard should be relatively even.

In [ ]:
import time
from tqdm import tqdm
genotype1 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
             torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
             torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
             torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
             ]
genotype2 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
             torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
             torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
             torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
             ]

genotype3 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
             torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
             torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
             torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
             ]

genotype4 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
             torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
             torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
             torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
             ]
genotype5 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
             torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
             torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
             torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
             ]
genotype6 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
             torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
             torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
             torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
             ]
scoreboard = [0,0,0,0]
for i in tqdm(range(75)):
  AI_1 = AI_Player(0,genotype1)
  AI_2 = AI_Player(1,genotype2)
  AI_3 = AI_Player(2,genotype3)
  AI_4 = AI_Player(3,genotype4)
  b = Board(AI_1,AI_2,AI_3,AI_4)
  while b.TakeTurn() != "Game Over":
    pass
  winner = b.winner.name
  scoreboard[int(winner)] += 1
print("\n\n",scoreboard)

# Creating a model training function

In [ ]:
import threading
from tqdm import tqdm

#This will play one game of monopoly and update scoreboard
def Monopoly_Round(glist,scoreboard):
  A1 = AI_Player(0,glist[0])
  A2 = AI_Player(1,glist[1])
  A3 = AI_Player(2,glist[2])
  A4 = AI_Player(3,glist[3])
  A5 = AI_Player(4,glist[4])
  A6 = AI_Player(5,glist[5])

  b1 = Board(A1,A2,A3,A4,A5,A6)

  while b1.TakeTurn() != "Game Over":
    pass
  scoreboard[int(b1.winner.name)] += 1

#This slightly changes the parameters for all networks in a genotype
def mutate(genotype, mrate = 0.1):
    for tensor in genotype:
      tensor += mrate * torch.randn_like(tensor)
      torch.clamp_(tensor,-1,1)
    return genotype

#This gives a chance for the other losing genotypes to crossover the parameters from the winning genotype
def crossover(winnerid,glist):
  for i in range(len(glist)):
    if i == winnerid:
      pass
    else:
      for j in range(len(glist[i])):
        if random.random() < 0.2:
          glist[i][j] = glist[winnerid][j]
      glist[i] = mutate(glist[i])

def Train_Model(num_generations):
  genotype1 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
               torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
               torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
               torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
              ]
  genotype2 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
              torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
              torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
              torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
              ]

  genotype3 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
              torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
              torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
              torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
              ]

  genotype4 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
              torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
              torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
              torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
              ]
  genotype5 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
              torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
              torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
              torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
              ]
  genotype6 = [torch.rand(14,5),torch.rand(1,5),torch.rand(5,3),torch.rand(1,3),torch.rand(3,1),torch.rand(1,1),
              torch.rand(4,2),torch.rand(1,2),torch.rand(2,1),torch.rand(1,1),
              torch.rand(13,10),torch.rand(1,10),torch.rand(10,6),torch.rand(1,6),torch.rand(6,6),torch.rand(1,6),
              torch.rand(14,4),torch.rand(1,4),torch.rand(4,1),torch.rand(1,1)
              ]
  glist = [genotype1,genotype2,genotype3,genotype4,genotype5,genotype6]
  for i in tqdm(range(num_generations)):
    scoreboard = np.zeros(6)
    for j in range(75):
      Monopoly_Round(glist,scoreboard)
    winnerid = np.argmax(scoreboard)
    crossover(winnerid,glist)
  best_geno_name = np.argmax(scoreboard)
  print("ScoreBoard: ",scoreboard)
  print("The winner is: ",best_geno_name)
  print("The genotype of the winner is: \n \n",glist[best_geno_name])
Train_Model(25)

